In [ ]:
# Install required packages if not already installed
#!pip install geemap --quiet
#!pip install earthengine-api --quiet

import os
import shutil
from pathlib import Path
from datetime import datetime, timedelta
import ee
import geemap
import geopandas as gpd
from shapely.geometry import box
import ipywidgets as widgets
from IPython.display import display
import math
import time
import json
from concurrent.futures import ThreadPoolExecutor, as_completed

In [ ]:
# Authenticate and initialize Earth Engine
ee.Authenticate()
ee.Initialize()

In [ ]:
main_directory_input = input("Enter the main directory path (e.g., C:\\user\\satellite_project): ")
main_directory = Path(main_directory_input)
main_directory.mkdir(parents=True, exist_ok=True)
shp_folder = main_directory.parent.parent / "SHP"
output_folder = main_directory.parent.parent / 'Sentinel2_Square_Exports'

In [ ]:
def get_s2_pixel_size(lat):
    meters_per_degree = 111320
    pixel_size_m = 10
    pixel_deg = pixel_size_m / (meters_per_degree * math.cos(math.radians(lat)))
    return pixel_deg

def make_square(lon, lat, n_pixels=512):
    px_deg = get_s2_pixel_size(lat)
    half_side = (n_pixels / 2) * px_deg
    xmin, xmax = lon - half_side, lon + half_side
    ymin, ymax = lat - half_side, lat + half_side
    return box(xmin, ymin, xmax, ymax)

In [ ]:
# Set up Sentinel-2 tile (see previous code for reference)
lake_tahoe_coords = [39.0968, -120.0324]
lake_tahoe_point = ee.Geometry.Point([-120.0324, 39.0968])
aoi = lake_tahoe_point.buffer(20000).bounds()
start = '2024-02-01'
end = '2024-02-28'

s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
      .filterBounds(aoi)
      .filterDate(start, end)
      .sort('CLOUDY_PIXEL_PERCENTAGE'))
img = s2.first()
date = ee.Date(img.get('system:time_start')).format('YYYY-MM-dd').getInfo()

vis_params = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000, 'gamma': 1.1}

Map = geemap.Map(center=lake_tahoe_coords, zoom=11)
Map.addLayer(img, vis_params, f"S2 TrueColor {date}")

# List to collect info for all clicked squares
clicked_squares = []

def handle_map_click(**kwargs):
    if kwargs.get('type') == 'click':
        coords = kwargs.get('coordinates')
        if coords is None:
            return
        lat, lon = coords[0], coords[1]
        square_geom = make_square(lon, lat)
        #timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        # Save info for export
        clicked_squares.append({
            'lon': lon,
            'lat': lat,
            'date': date,  # Use the image's date, not system time!
            'geometry': square_geom
        })
        # Show on map
        gdf = gpd.GeoDataFrame([{'geometry': square_geom}], crs='EPSG:4326')
        Map.add_gdf(gdf, layer_name=f"Square {len(clicked_squares)}", style={'color': 'red', 'fillOpacity': 0})
        
Map.on_interaction(handle_map_click)
Map

In [ ]:
def export_squares_to_shapefile(_):
    if len(clicked_squares) == 0:
        print("No squares to export!")
        return
    # Remove and re-create SHP folder
    if shp_folder.exists():
        shutil.rmtree(shp_folder)
    shp_folder.mkdir(parents=True, exist_ok=True)
    shp_path = shp_folder / "clicked_squares.shp"
    
    gdf = gpd.GeoDataFrame(clicked_squares, geometry='geometry', crs="EPSG:4326")
    gdf["Id"] = range(1, len(gdf) + 1)
    gdf = gdf[["Id", "lon", "lat", "date", "geometry"]]
    gdf.to_file(shp_path)
    print(f"Saved {len(gdf)} squares to {shp_path}")

export_button = widgets.Button(
    description="Export All Squares to Shapefile",
    button_style='success',
    layout=widgets.Layout(width='300px')  # Make button wide enough
)
export_button.on_click(export_squares_to_shapefile)
display(export_button)

In [ ]:
# ---- Settings ----
output_folder.mkdir(exist_ok=True)
scale = 10  # meters (S2 bands 1-9, 11, 12 are 10/20/60m; use 10m for all)
bands_s2 = ['B1','B2','B3','B4','B5','B6','B7','B8','B8A','B9','B11','B12']
max_workers = 8

# ---- Load your SHP ----
shp_path = shp_folder / "clicked_squares.shp"
gdf = gpd.read_file(shp_path)

def safe_ee_getinfo(obj, retries=4):
    delay = 20
    for i in range(retries):
        try:
            return obj.getInfo()
        except Exception as e:
            print(f"getInfo error ({i+1}/{retries}): {e}. Retrying in {delay}s…")
            time.sleep(delay)
            delay *= 2
    raise RuntimeError("Failed to getInfo after retries.")

def safe_export(image, filename, scale, region, bands):
    for attempt in range(4):
        try:
            geemap.ee_export_image(
                image.select(bands),
                filename=filename,
                scale=scale,
                region=region,
                file_per_band=False
            )
            return True
        except Exception as e:
            print(f"Error exporting {filename} (attempt {attempt+1}/4): {e}")
            time.sleep(20 * (2 ** attempt))
    print(f"Failed export after 4 attempts: {filename}")
    return False

def export_square_row(row):
    square_geom = row['geometry']
    coords = list(square_geom.exterior.coords)
    roi = ee.Geometry.Polygon([list(coords)])
    date = row['date'][:10]  # 'YYYY-MM-DD'
    sq_id = row['Id']

    # Use a one-day window
    date_obj = datetime.strptime(date, "%Y-%m-%d")
    next_day = (date_obj + timedelta(days=1)).strftime("%Y-%m-%d")
    
    # Find the S2 image for the *exact* day window
    s2 = (ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")
          .filterBounds(roi)
          .filterDate(date, next_day)
          .sort('CLOUDY_PIXEL_PERCENTAGE')
         )
    img = s2.first()
    try:
        info = img.getInfo()
    except Exception as e:
        print(f"No Sentinel-2 image found for Square Id {sq_id} on {date}. Skipping.")
        return False

    tif_name = f"Square_{sq_id}_Sentinel2_{date}.tif"
    tif_path = str(output_folder / tif_name)
    region_geo = safe_ee_getinfo(roi)
    region_coords = region_geo['coordinates']
    print(f"Exporting {tif_name} ...")
    return safe_export(img, tif_path, scale, region_coords, bands_s2)

# ---- Main Export Routine ----
start_time = time.time()
print(f"Starting Sentinel-2 download for {len(gdf)} squares…")

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [executor.submit(export_square_row, row) for _, row in gdf.iterrows()]
    for f in as_completed(futures):
        result = f.result()
        if not result:
            print("An export failed or no image was available for one square.")

elapsed = time.time() - start_time
print(f"\nAll exports done. Elapsed time: {elapsed/60:.1f} min. Files saved in: {output_folder}")